In [ ]:
import pandas as pd, numpy as np,xarray as xr
from pathlib import Path
import re, yaml, copy, json
import helper, events_methods
from helper import RenderJSON

In [ ]:
import itables
itables.init_notebook_mode(all_interactive=True )
itables.options.maxBytes = "1MB"
itables.options.lengthMenu = [25, 10, 50, 100, 200]
itables.options.buttons = ["copyHtml5", "csvHtml5", "excelHtml5"]
# itables.options.scrollY="200px"
# itables.options.scrollCollapse=True
# itables.options.paging=False
# itables.options.column_filters = "footer"
itables.options.layout={"topEnd": "pageLength", "top1": "searchBuilder"}

In [ ]:
params = yaml.safe_load(Path("params.yaml").open("r"))
event_path = Path(params["fiberevents_path"])
info_path = Path(params["config_path"])
res_events_path = Path(params["dest_path"])
RenderJSON(params)

In [ ]:
event_df = pd.read_csv(event_path, sep=",").rename(columns=dict(Name="channel_name"))
event_df.insert(0, "t", event_df.pop("TimeStamp")/1000)
event_df

In [ ]:
info = yaml.safe_load(info_path.open("r"))
info

In [ ]:
event_spec = events_methods.FiberEventProcessing.process_info(event_df["channel_name"].drop_duplicates().to_list(), info["processing"])
pd.DataFrame(list(event_spec.values()))

In [ ]:
all=[]
for ev_name, item in event_spec.items():
    ev_dataframe = events_methods.FiberEventProcessing.compute_evdataframe(event_df, item)
    if len(ev_dataframe.index) == 0: continue
    events = events_methods.FiberEventProcessing.call(item["method"],ev_dataframe, item)
    if len(events.index)!=0:
        all.append(events)
all = pd.concat(all).sort_values("t")[["event_name", "t", "duration", "n_segments","metadata", "waveform_changes", "waveform_values"]]
all

In [ ]:
if "display" in info and "rename" in info["display"]:
    all["event_name"] = all["event_name"].map(lambda e: info["display"]["rename"][e] if e in info["display"]["rename"] else e)
json_cols = ["metadata", "waveform_changes", "waveform_values"]
for col in json_cols:
    all[f"{col}_json"] = all[col].apply(lambda d: json.dumps(d))
all.drop(columns=json_cols).to_csv(res_events_path, sep="\t", index=False)
reloaded = pd.read_csv(res_events_path, sep="\t", index_col=False)
for col in reloaded.columns:
    if col.endswith("_json"):
        reloaded[col[:-5]] = reloaded.pop(col).apply(lambda s: json.loads(s) if not pd.isna(s) else None)
reloaded

In [ ]:
summary = events_methods.EventProcessing.summarize(reloaded, precision=0.02, min_score=0.5)
summary